# Exercise 9
Olivier Kanamugire

## Task 1 - Linear Triangulation with DLT (2 points)

In [1]:
import numpy as np
import h5py

#### (a) Load the fundamental matrix F and points from task1.mat

In [2]:
# load file for Task 1
with h5py.File('task1.mat', 'r') as file:
    data = {key: np.array(file[key]).T for key in file.keys()}

F = data["F"]
points_l = np.concatenate([data["points_l"], np.ones((1, data["points_l"].shape[1]))], axis=0)
points_r = np.concatenate([data["points_r"], np.ones((1, data["points_r"].shape[1]))], axis=0)

In [3]:
F

array([[ 1.92914000e-06,  3.85751891e-06, -1.75411815e-03],
       [-7.88234869e-06,  4.28121773e-06,  1.05959161e-02],
       [-2.10406355e-03, -1.10758937e-02,  7.38170021e-01]])

#### (b) Create camera matrices ML and MR by using the definitions from Slide 26 of the lecture slides 8.

The camera matrices for the left and right cameras are defined as:

$$
M_L = \begin{bmatrix} I &| 0 \end{bmatrix}
$$

$$
M_R = \begin{bmatrix} [e_R]_\times F &| e_R \end{bmatrix}
$$

where $[e_R]_\times$ is the skew-symmetric matrix corresponding to the epipole vector \( e_R = (e_1, e_2, e_3)^T \):

$$
[e]_\times = \begin{bmatrix} 
0 & -e_3 & e_2 \\
e_3 & 0 & -e_1 \\
-e_2 & e_1 & 0
\end{bmatrix}
$$


In [4]:
# compute epipoles
U, S, Vt = np.linalg.svd(F.T)
# Epipole in the right image
eR = Vt[-1]  
# Normalize
eR /= eR[-1]

U, S, Vt = np.linalg.svd(F)
# Epipole in the left image
el = Vt[-1]  
# Normalize
el /= el[-1]  

print('The right epipole is: ', eR)
print('The left epipole is: ', el)

The right epipole is:  [2.49091090e+03 3.42696372e+02 1.00000000e+00]
The left epipole is:  [ 1.25134452e+03 -1.71068668e+02  1.00000000e+00]


In [5]:
# Compute skew-symmetric matrix of eR
eR_x = np.array([
    [0, -eR[2], eR[1]],
    [eR[2], 0, -eR[0]],
    [-eR[1], eR[0], 0]
])
eR_x

array([[ 0.00000000e+00, -1.00000000e+00,  3.42696372e+02],
       [ 1.00000000e+00,  0.00000000e+00, -2.49091090e+03],
       [-3.42696372e+02,  2.49091090e+03,  0.00000000e+00]])

In [6]:
# find camera matrices Mr and Ml
Ml = np.hstack((np.eye(3), np.zeros((3, 1))))  # M_L = [I | 0]
Mr = np.hstack((eR_x @ F, eR.reshape(3, 1)))  # M_R = [ [eR]_x * F | eR ]

# Print results
print("Left Camera Matrix (Ml):\n", Ml)
print("\nRight Camera Matrix (Mr):\n", Mr)


Left Camera Matrix (Ml):
 [[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]

Right Camera Matrix (Mr):
 [[-7.21047062e-01 -3.79567286e+00  2.52957592e+02  2.49091090e+03]
 [ 5.24103676e+00  2.75890682e+01 -1.83871751e+03  3.42696372e+02]
 [-2.02953376e-02  9.34217419e-03  2.69946128e+01  1.00000000e+00]]


#### c) LINEAR TRIANGULATION

In [7]:
def triangulate_point(pL, pR, ML, MR):
    A = np.array([
        pL[0] * ML[2] - ML[0], 
        pL[1] * ML[2] - ML[1], 
        pR[0] * MR[2] - MR[0],  
        pR[1] * MR[2] - MR[1]   
    ])
    
    # Solve AP = 0 using SVD
    _, _, Vt = np.linalg.svd(A)
    P = Vt[-1]  

    # Normalize the homogeneous coordinate
    P = P / P[-1]  

    return P[:3]  

#### Looping 

In [8]:
def triangulate_points(points_l, points_r, ML, MR):
    num_points = points_l.shape[1]
    P_3D = np.zeros((3, num_points))  # Store triangulated 3D points

    for i in range(num_points):
        P_3D[:, i] = triangulate_point(points_l[:, i], points_r[:, i], ML, MR)

    return P_3D

P_3D = triangulate_points(points_l, points_r, Ml, Mr)
print("Triangulated 3D points:\n", P_3D)

Triangulated 3D points:
 [[1.32784822e+02 9.53824490e+01 8.86565301e+01 9.79170834e+01
  8.96404093e+01 7.83697189e+01 1.40212593e+02 1.29959924e+02
  7.86528675e+01]
 [1.08760522e+02 1.90363973e+01 2.43472769e+01 6.64050071e+01
  7.51157081e+01 6.09488470e+02 3.19403470e+02 1.32303905e+02
  5.84577233e+01]
 [5.32871118e-01 3.79732883e-01 4.77246695e-01 5.24729204e-01
  9.28189804e-01 4.11747887e+00 1.59633442e+00 6.44138792e-01
  1.08222805e+00]]


In [9]:
# compute and print the error
def project_points(P_3D, M):
    # Convert to homogeneous coordinates (4, N)
    P_homogeneous = np.vstack((P_3D, np.ones((1, P_3D.shape[1]))))

    # Project onto image plane
    projected_2D = M @ P_homogeneous  

    # Normalize 
    projected_2D /= projected_2D[2]  

    return projected_2D[:2] 

def compute_reprojection_error(P_3D, points_l, points_r, ML, MR):
    # Project 3D points back to 2D
    projected_l = project_points(P_3D, ML) 
    projected_r = project_points(P_3D, MR)  

    # Compute squared error
    error_l = np.sum((points_l[:2] - projected_l) ** 2)  
    error_r = np.sum((points_r[:2] - projected_r) ** 2)

    total_error = error_l + error_r

    return total_error

# Compute the reprojection error
error = compute_reprojection_error(P_3D, points_l, points_r, Ml, Mr)
print("Reprojection Error:", error)

Reprojection Error: 0.000520570040926605


In [12]:
projected_l = project_points(P_3D, Ml) 
projected_r = project_points(P_3D, Mr)  

error_l = [np.sum((points_l[:2] - projected_l) ** 2) ]
error_r = [np.sum((points_r[:2] - projected_r) ** 2)]

##### The reprojection error is negligible, even when evaluated using the L2 norm. This indicates that the projected points are very close to the actual points.